In [1]:
query = '''
SELECT
  visitId 
  , STRING_AGG(p.productSKU ORDER BY h.hitNumber) AS product_sequence
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`, UNNEST(hits) as h, UNNEST(h.product) as p
WHERE _TABLE_SUFFIX BETWEEN '20160801' AND '20160807'
  AND p.isClick IS NOT NULL
GROUP BY visitId
'''

In [2]:
import pandas as pd
from gensim.models import Word2Vec
from google.oauth2.service_account import Credentials

In [3]:
class ProductSequences:
    '''
    ProductSequences
    '''
    def __init__(self, df_column):
        df_column = df_column.copy()
        self.sequences = df_column.values.tolist()

    def __iter__(self):
        for sequence in self.sequences:
            yield sequence.split(',')

In [5]:
df = pd.read_gbq(query,
                 credentials=Credentials.from_service_account_file('./private-key.json'))

print(df.info())

df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3019 entries, 0 to 3018
Data columns (total 2 columns):
visitId             3019 non-null int64
product_sequence    3019 non-null object
dtypes: int64(1), object(1)
memory usage: 47.3+ KB
None


,visitId,product_sequence
0,1470193777,GGOEGBRJ037399
1,1470146349,GGOEGBRA037499
2,1470126740,GGOEGBRA037499
3,1470183593,GGOEAFKQ020599
4,1470193792,GGOEGBRJ037399


In [6]:
model = Word2Vec(ProductSequences(df.product_sequence), min_count=10, sg=1)

In [7]:
embeddings_dict = dict(zip(model.wv.index2word, model.wv.vectors))